In [20]:
import pandas as pd
import numpy as np
import dataclean as dc 
from pandas import DataFrame
from sklearn.preprocessing import StandardScaler, MinMaxScaler


/home/taoyida/miniconda3/envs/DMT/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#若想本地运行，此处需要修改
data=pd.read_csv('training_set_VU_DM.csv')

KeyboardInterrupt: 

clean data

In [3]:
def Median_padding(data:DataFrame,column:str):
    # 计算众数并确保众数计算成功
    if not data[column].mode().empty:
        mode_value = data[column].mode()[0]
        print(column+"众数值:", mode_value)

        # 用众数填充空缺值
        data[column].fillna(mode_value, inplace=True)

        # 检查填充后的空缺值情况
        print(column+"填充后的空缺值个数:", data[column].isnull().sum())
    else:
        print(column+"无法计算众数，因为数据列中没有有效值。")
        

def fill_missing_values(data:DataFrame):#严格按照每个酒店的中位数进行填充，但是太慢了，放弃
    prop_id=data['prop_id'].unique()
    fill_cols=['comp1_rate','comp1_inv','comp1_rate_percent_diff',
                'comp2_rate','comp2_inv','comp2_rate_percent_diff',
                'comp3_rate','comp3_inv','comp3_rate_percent_diff',                
                'comp4_rate','comp4_inv','comp4_rate_percent_diff',
                'comp5_rate','comp5_inv','comp5_rate_percent_diff',
                'comp6_rate','comp6_inv','comp6_rate_percent_diff',
                'comp7_rate','comp7_inv','comp7_rate_percent_diff',
                'comp8_rate','comp8_inv','comp8_rate_percent_diff']
    #用一个字典存储每个属性的中位数
    comp_median = {col: data[col].median() for col in fill_cols}
    
    temp = data[['prop_id'] + fill_cols].copy()

    

    def process_group(prop):
        prop_data = temp[temp['prop_id'] == prop]
        for col in fill_cols:
            if prop_data[col].isna().all():  # 如果这一列全是空值,则用comp_median[col]填充
                fill_value = comp_median[col]
            else:  # 如果这一列不全是空值,则用这一列的中位数填充
                fill_value = prop_data[col].median()
            temp.loc[temp['prop_id'] == prop, col] = prop_data[col].fillna(fill_value)
    
    # 使用 Parallel 和 delayed 并行处理
    Parallel(n_jobs=-1,backend='threading',verbose=1)(delayed(process_group)(prop) for prop in prop_id)
    
    data.update(temp)


        



In [ ]:
fill_cols=['comp1_rate','comp1_inv','comp1_rate_percent_diff',
                'comp2_rate','comp2_inv','comp2_rate_percent_diff',
                'comp3_rate','comp3_inv','comp3_rate_percent_diff',                
                'comp4_rate','comp4_inv','comp4_rate_percent_diff',
                'comp5_rate','comp5_inv','comp5_rate_percent_diff',
                'comp6_rate','comp6_inv','comp6_rate_percent_diff',
                'comp7_rate','comp7_inv','comp7_rate_percent_diff',
                'comp8_rate','comp8_inv','comp8_rate_percent_diff',
                'visitor_hist_starrating','visitor_hist_adr_usd','prop_review_score',
                'prop_location_score2','srch_query_affinity_score','orig_destination_distance','gross_bookings_usd']

for col in fill_cols:
    Median_padding(data,col)


feature engineering

处理时间变量

In [16]:
data['date_time'] = pd.to_datetime(data['date_time'])
# 提取基本成分
data['year'] = data['date_time'].dt.year
data['month'] = data['date_time'].dt.month
data['day'] = data['date_time'].dt.day
data['hour'] = data['date_time'].dt.hour
data['minute'] = data['date_time'].dt.minute
data['second'] = data['date_time'].dt.second

# 提取衍生特征
data['day_of_week'] = data['date_time'].dt.dayofweek  # 星期几，0是周一，6是周日
data['is_weekend'] = data['day_of_week'] >= 5  # 判断是否是周末
data['quarter'] = data['date_time'].dt.quarter  # 季度

处理布尔型变量

In [ ]:
print(df)